<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第４章１節：部分構造で絞り込む話" data-toc-modified-id="第４章１節：部分構造で絞り込む話-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>第４章１節：部分構造で絞り込む話</a></span></li><li><span><a href="#活性化合物のデータセットでビフェニルを探す" data-toc-modified-id="活性化合物のデータセットでビフェニルを探す-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><u><b>活性化合物のデータセットでビフェニルを探す</b></u></a></span></li><li><span><a href="#ライブラリ化合物で部分構造による絞り込み" data-toc-modified-id="ライブラリ化合物で部分構造による絞り込み-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><u><b>ライブラリ化合物で部分構造による絞り込み</b></u></a></span></li><li><span><a href="#部分構造検索をKNIMEで" data-toc-modified-id="部分構造検索をKNIMEで-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><u><b>部分構造検索をKNIMEで</b></u></a></span></li><li><span><a href="#オルト位置換の認識に失敗" data-toc-modified-id="オルト位置換の認識に失敗-5"><span class="toc-item-num">5&nbsp;&nbsp;</span><u><b>オルト位置換の認識に失敗</b></u></a></span></li><li><span><a href="#まとめ" data-toc-modified-id="まとめ-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><u><b>まとめ</b></u></a></span></li></ul></div>

# 第４章１節：部分構造で絞り込む話

Twitterで拝見した以下の処理の手順を参考に、指標を用いた粗い絞り込みまでを実施しました。

**処理の手順**

||処理|意図|
|:---:|:---:|:---:|
|①|分子量の範囲を設定|粗い絞り込み|
|②|部分構造で絞り込み|活性化合物を参照したLBVS（？）|
|③|粗いドッキング|タンパク質との相互作用を予測するSBVS（？）|
|④|ファーマコフォアを指定したより詳細なドッキング|SBVS ２回目（？）|

**指標による絞り込み基準**

|指標|分子量|LogP|水素結合供与体数|水素結合受容体数|回転可能結合数|極性表面積|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|Rule of 5|≦500|≦5|≦5|≦10|||
|基準|>300|>3||>3|>3|>60|


今回は指標による絞り込みの次の段階、部分構造での絞り込みを行いたいと思います。  
参考にさせていただいている処理の方法では、**オルト位に置換基の入ったビフェニル**で絞り込んだとのことでしたが、いきなり置換基なども含めた処理を行うのは難しいので、まずはビフェニル構造での絞り込みを行いたいと思います。


# <u><b>活性化合物のデータセットでビフェニルを探す</b></u>

そもそもなぜビフェニルなのか？　活性化合物のデータセット （[共闘用シェアデータ](https://github.com/masahiro-mochizuki/pd1_inhibitor_dataset) ）中の分子（マクロサイクル型を除く）が、ビフェニル構造をもつのか検証してみます。

「化学の新しいカタチ」さんのこちらの記事「 [RDKitを用いた部分構造検索とMCSアルゴリズム](https://future-chem.com/substruct-search-mcs/) 」 を参考にRDKitの部分構造検索メソッドを利用します。

まずは必要なものをimport・・・


```python
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw
```

RDKitで部分構造検索を行う際、検索構造（query）はMolオブジェクトにする必要があるということなので、まずはビフェニル構造のMolオブジェクトをSMILESから作成します。



```python
biphenyl = Chem.MolFromSmiles('c1ccc(cc1)c1ccccc1')
Draw.MolToImage(biphenyl)
```


<img src="./images/4-1_image1.png">


きちんとビフェニル構造のMolオブジェクトが作られていそうです。こちらをテンプレートとして用い、部分構造検索を行います。


```python
#以前取り出した鎖状分子のSDFを使用する
chain_compounds_suppl = Chem.ForwardSDMolSupplier('chain_compounds.sdf')

mols = [mol for mol in chain_compounds_suppl if mol is not None]   
```


**HasSubstructMatch**をもちいて、ビフェニル構造を持つ分子と持たない分子を区別し、それぞれ別々のリストに入れていきます。



```python
#ビフェニルありのリスト
biphenyl_compounds = []
#ビフェニルなしのリスト
w_o_biphenyl = []

for mol in mols:
    if mol.HasSubstructMatch(biphenyl):
        biphenyl_compounds.append(mol)
    else:
        w_o_biphenyl.append(mol)

print('Number of compounds with biphenyl', len(biphenyl_compounds))
# Number of compounds with biphenyl 29
print('Number of compounds without biphenyl', len(w_o_biphenyl))
# Number of compounds without biphenyl 10
```


39個の分子のうち、29個にビフェニルが含まれました。確かに活性化合物は多くがビフェニル構造を持つようです。

最初の一つの分子を取り出して、ビフェニル構造の確認を行います。

**GetSubstructMatches**でマッチした複数の原子インデックスを取得できるそうです。


```python
test_mol = biphenyl_compounds[0]
matched_atoms = test_mol.GetSubstructMatches(biphenyl)
print(matched_atoms)
# ((23, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29),)
```

原子インデックスのタプルのタプルとして得られました。

原子インデックスを描画してみます。

RDKitのメーリングリストの[ディスカッション](https://sourceforge.net/p/rdkit/mailman/message/36208102/) によると、オプションを「**Draw.DrawingOptions.includeAtomNumbers=True**」と設定することで、表示できるようです。少し書き方を変えて以下のようにしました。


```python
Draw.MolToImage(test_mol, includeAtomNumbers = True)
```

<img src="./images/4-1_image2.png">

構造式の上の方に、認識された原子インデックスがあるのがわかります。

こちらをハイライトしてみます。


```python
#タプルのタプルなのでインデックス[0]が必要
Draw.MolToImage(test_mol, highlightAtoms = matched_atoms[0]) 
```


<img src="./images/4-1_image3.png">

順番に眺めます。


```python
from ipywidgets import interact,fixed,IntSlider
import ipywidgets

def biphenyl_viewer(idx):
    mol = biphenyl_compounds[idx]
    matched_atoms = mol.GetSubstructMatches(biphenyl)
    return(Draw.MolToImage(mol, highlightAtoms = matched_atoms[0]))

interact(biphenyl_viewer, idx=ipywidgets.IntSlider(min=0,max=len(biphenyl_compounds)-1, step=1));
```

<img src="./images/4-1_gif1.gif">

ビフェニル構造は末端近くにあるものが多いようですが、いくつかは分子の中央部分にもあります。共通構造といっても分子の中で占める位置が異なるものもあるようです。  
中央部分にビフェニルを含むものは、ビフェニルを中心とした対称構造のようにもみえるので、結合様式等（？）の知見を反映してデザインされた化合物なのかもしれません。  


# <u><b>ライブラリ化合物で部分構造による絞り込み</b></u>

それでは、ライブラリ化合物の中からビフェニル構造を持つ化合物を取り出してみたいと思います。

まずは指標を計算済みの**Enamine_Premium_collection**について、指標でのフィルタリングを実施します。



```python
EPc_suppl = Chem.ForwardSDMolSupplier('./Enamine_Premium_collection.sdf')

EPc_mols_filt = []

for x in EPc_suppl:
    if x is not None:
        mol = x
    MW_value = mol.GetDoubleProp('MW')
    MolLogP_value = mol.GetDoubleProp('MolLogP')
    NumHDonors_value = mol.GetDoubleProp('NumHDonors')
    NumHAcceptors_value = mol.GetDoubleProp('NumHAcceptors')
    NumRotatableBonds_value = mol.GetDoubleProp('NumRotatableBonds')
    TPSA_value = mol.GetDoubleProp('TPSA')
    
    #フィルタリング基準を満たす場合のみリストに追加する
    if MW_value > 300 and MW_value <= 500 \
    and MolLogP_value > 3 and MolLogP_value <=5\
    and NumHDonors_value <= 5 \
    and NumHAcceptors_value > 3 and NumHAcceptors_value <= 10 \
    and NumRotatableBonds_value > 3 \
    and TPSA_value > 60 :
        filtered_mols.append(mol)
```


フィルタリングを満たした化合物のリストを、活性化合物群に対して行なったのと同様にビフェニルの有無で別々のリストにします。


```python
biphenyl = Chem.MolFromSmiles('c1ccc(cc1)c1ccccc1')
biphenyl_compounds = []
w_o_biphenyl = []
​
for mol in filtered_mols:
    if mol.HasSubstructMatch(biphenyl):
        biphenyl_compounds.append(mol)
    else:
        w_o_biphenyl.append(mol)

print(len(biphenyl_compounds))  
#12
print(len(w_o_biphenyl))
#4048
```



**Enamine_Premium_collection**で条件を満たすものは12個となりました。
構造を眺めてみます。



```python
Draw.MolsToGridImage(biphenyl_compounds, molsPerRow=4, subImgSize=(200,200))
```


<img src="./images/4-1_image4.png">

なかなかPremium感のあるいい感じの化合物が残っている気がします。

他のライブラリに適用した結果とあわせてまとめます。

||Enamine_Premium|Enamie_Advenced|Enamine_HTS|UOS_HTS|total|
|:---:|:---:|:---:|:---:|:---:|:---:|
|指標でのフィルタリング後|4060|37431|414562|106948|563001|
|ビフェニル有|12|329|3182|697|4220 <br> (0.7%)|
|ビフェニル無|4048|37102|411380|106251|558781 <br> (99.3%)|

全体を合わせて 4220 個の分子が残りました。まだまだ多いですが、かなり数を絞り込めてきているように思います。


# <u><b>部分構造検索をKNIMEで</b></u>

Twitter話題のKNIME・・・、こちらのブログ[非プログラマーのためのインフォマティクス 入門。（仮）](http://sumtat.hatenablog.com) のコンテンツが素晴らしいのでKNIMEブームに便乗することにしました。

（ブログタイトルがまさにこれを求めていた！！って感じです）

今回行った流れは、

1. 活性化合物群をビフェニルの有無で分ける
2. フィンガープリントを計算し、PCAで次元圧縮したうえでプロット
3. インタラクティブに遊ぶ

です。

以下の記事を参考にさせていただきました

1. [化合物をベクトルにして比較しプロットする](https://qiita.com/Mochimasa/items/f1b60246ece7da46f6a9)
2. [KNIMEで化合物をクラスタリング＆可視化してみよう](http://sumtat.hatenablog.com/entry/2017/09/18/095508)
3. [Make interactive plot with Knime](https://iwatobipen.wordpress.com/2018/12/14/make-interactive-plot-with-knime-rdkit-chemoinformatics-knime/)
4. [【KNIME】CellProfilerの画像解析結果を可視化する](https://t-kahi.hatenablog.com/entry/2019/01/05/100423)


早速ワークフロー全体です。２通りの可視化を行うため、プロットのためののメタノード（灰色）が２つあります。

<img src="./images/4-1_image5.png">

KNIMEの視認性の高さはすごいですね！何となくやっていることがわかります。
Jupyter NoteBookは３日経つと自分の操作の意味すらわからなくなります（私だけ？）が、これなら思い出しやすそうです。

一応ワークフローの説明をすると

1. SDFの読み込み(SDF Reader)
2. SDFの構造ををRDKitのMolオブジェクトに変換（RDKit To Molecule）
3. ビフェニル構造の有無で２つに分割（RDKit Substructure Filter）
4. 分割したテーブルにクラス名のカラムを追加（Constant Value Column）
5. ２つのテーブルを一つに再結合（Concatenate）
6. フィンガープリント（Morgan/ 1024ビット/ 半径2）を計算（RDKit Fingerprint）
7. フィンガープリントのビットベクトルをPCAに渡すため整数値のカラムに分割（Expand Bit Vector）
8. PCA（PCA）
9. 分子の構造を描画のためSVGに変換（Renderer to Image）
10. クラスに応じた色分け（Color Manager）
11. プロット

プロットでは、**ビフェニルの有無**での色分けと、各化合物の出元（**特許の出願人（applicant）**）での色分けの２通りを行っています。
前者では**Tile View** (KNIME 3.7.1 ではCard Viewの名称が変更となったそうです)、後者では**Table View**を使用しました。

メタノードの中身はこちら

<img src="./images/4-1_image6.png"> <img src="./images/4-1_image7.png">

実行した結果はこのようになります。
まずはビフェニルの有無で色分けした場合・・・

<img src="./images/4-1_gif2.gif">

ビフェニルを含まない構造（赤色）の多くは、プロットの右下に集まっています。これらは全て**applicant**が**Aurigene**となっています。他のapplicantとは系統の異なる化合物を報告しているようです。
また、ビフェニルを含む構造（緑色）でも左下には**中国医学科学院**の化合物群、中央上部には**Incyte**の化合物群が集まっているように見えます。興味深いことに、Morganフィンガープリントにもとづくプロットで化合物の出所がうまく分かれているようです。

**applicant**による色分けを眺めてみます。

<img src="./images/4-1_gif3.gif">

先のプロットで想定した通り、applicantごとにプロット上でうまく分かれていそうです。

見よう見まねでも何となく使えてしまうとはKNIMEの完成度高い！


# <u><b>オルト位置換の認識に失敗</b></u>

元々の部分構造フィルタリングの目標は、**オルト位に置換基の入ったビフェニル**だったので、SMILESのワイルドカード（\*）をつかって、以下のコードを実施しました。


```python
ortho_biphenyl = Chem.MolFromSmiles('c1ccc(c(*)c1)c1ccccc1')
Draw.MolToImage(ortho_biphenyl)
```

<img src="./images/4-1_image8.png">

オルト位置換（なんでも良い）が再現できていそうです。
以下のテスト分子で試します。

<img src="./images/4-1_image9.png">


```python
test_mol.HasSubstructMatch(ortho_biphenyl)
# Flase
```


・・・ダメでした。置換基あるのに・・・


# <u><b>まとめ</b></u>

今回、「ビフェニル構造を部分構造としてもつか否か」を基準としてライブラリのさらなる絞り込みを実施しました。その結果、約4000 個にまで化合物数を絞り込むことができました。また、KNIMEをもちいた可視化を実施し、フィンガープリントを持ちいることで化合物群を出所（出願者）に応じてうまく識別できる可能性があることがわかりました。

残念ながら、オルト位に置換基の入ったビフェニル、というフィルタリングはうまくできませんでした。私、応用力無さすぎ・・・
